In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [4]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])

# GAN Model

In [7]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.052149,0.152866,1.020008,3.057312
1,0.123697,0.229275,1.572754,4.585508
2,0.121998,0.224301,1.561952,4.486017
3,0.062488,0.192243,1.117595,3.844851
4,0.124140,0.252486,1.575309,5.049725
5,0.034784,0.124183,0.833122,2.483661
6,0.107623,0.224102,1.466874,4.482047
7,0.047333,0.157143,0.972630,3.142854
8,0.187382,0.269006,1.935102,5.380119
9,0.042565,0.149225,0.922446,2.984504


MEDIAN:
MSE                   0.085055
MAE                   0.208172
Euclidean Distance    1.292234
Manhattan Distance    4.163449
dtype: float64
0.20817244485737757


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [13]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [[1,1],[1,0.1],[1,0.01],[0.1,1],[0.1,0.1],[0.1,0.01],[0.01,1],[0.01,0.1],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [15]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

In [27]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Catboost','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Catboost,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.14426,1.23929,0.17708,0.17066,0.18393
1,1.00,0.10,0.14426,1.06144,0.22779,0.27995,0.08081
2,1.00,0.01,0.14426,1.04272,0.27345,0.31993,0.07639
3,0.10,1.00,0.14426,0.90755,0.21637,0.18107,0.21889
4,0.10,0.10,0.14426,0.33531,0.26273,0.21763,0.09530
5,0.10,0.01,0.14426,0.34894,0.29566,0.33063,0.04127
6,0.01,1.00,0.14426,0.74672,0.17123,0.16486,0.14193
7,0.01,0.10,0.14426,0.29085,0.24500,0.24257,0.29860
8,0.01,0.01,0.14426,0.30009,0.27688,0.29899,0.02710


## ABC Pre-generator - Stats 


In [29]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [30]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.29031,1.25478,0.88258,0.18901,0.11431
1,1,1,0.44975,1.23064,0.87832,0.35030,0.18303
2,1,1,0.42078,1.42587,0.98379,0.19155,0.25245
3,1,1,0.43359,1.24520,1.00000,0.31754,0.36813
4,1,1,0.36823,1.10807,0.89985,0.35659,0.17494
5,1,1,0.34959,1.06025,0.99644,0.22893,0.23016
6,1,1,0.40612,1.07032,1.00000,0.25452,0.27736
7,1,1,0.47881,1.22904,0.97343,0.35585,0.20099
8,1,1,0.29692,1.26820,0.96820,0.20240,0.20051
9,1,1,0.32433,1.36152,0.99623,0.15719,0.23160


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.387177
ABC pre-generator MAE            1.237921
Skip Node weight                 0.978611
ABC GAN MAE                      0.241726
ABC_GAN MAE (skip connection)    0.215575
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,1,0.31186,0.99195,0.98998,0.23428,0.22337
1,0.1,1,0.43040,0.72982,1.00000,0.52050,0.31388
2,0.1,1,0.35791,1.05063,0.99555,0.22909,0.19770
3,0.1,1,0.42437,0.79663,0.95032,0.31949,0.16097
4,0.1,1,0.25302,0.95273,0.98694,0.28690,0.17581
5,0.1,1,0.36768,0.84921,0.99555,0.36063,0.38644
6,0.1,1,0.43353,1.12283,0.86891,0.30905,0.14942
7,0.1,1,0.37974,0.97706,0.90165,0.26076,0.25680
8,0.1,1,0.30989,0.81079,0.85959,0.28656,0.13385
9,0.1,1,0.35632,1.03301,0.91324,0.32711,0.09685


Variance                         0.100000
Bias                             1.000000
Prior Model MAE                  0.362792
ABC pre-generator MAE            0.964899
Skip Node weight                 0.968629
ABC GAN MAE                      0.297976
ABC_GAN MAE (skip connection)    0.186758
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,1,0.38728,1.09869,0.93346,0.21018,0.11905
1,0.01,1,0.53611,1.03067,0.91697,0.36758,0.16454
2,0.01,1,0.43641,0.76017,0.99795,0.23458,0.21013
3,0.01,1,0.46893,1.05125,0.95772,0.26912,0.25218
4,0.01,1,0.41863,0.85753,0.99077,0.39182,0.18677
5,0.01,1,0.52122,0.71846,0.97956,0.38481,0.24568
6,0.01,1,0.48707,1.00235,0.99346,0.39385,0.29901
7,0.01,1,0.25713,0.71231,0.94414,0.22267,0.11580
8,0.01,1,0.39914,0.78874,0.99098,0.29706,0.20774
9,0.01,1,0.29066,0.89550,1.00000,0.28060,0.27047


...

Variance                         1.000000
Bias                             0.010000
Prior Model MAE                  0.375113
ABC pre-generator MAE            1.033574
Skip Node weight                 0.299942
ABC GAN MAE                      0.279367
ABC_GAN MAE (skip connection)    0.259238
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.01,0.51085,0.50383,0.13876,0.35834,0.22991
1,0.1,0.01,0.38288,0.39016,0.24636,0.23741,0.18503
2,0.1,0.01,0.35142,0.31922,0.13227,0.30161,0.19806
3,0.1,0.01,0.30469,0.34309,0.19530,0.23678,24.90852
4,0.1,0.01,0.28459,0.31449,0.10088,0.31357,12.18260
5,0.1,0.01,0.59744,0.56481,0.17860,0.24711,0.34491
6,0.1,0.01,0.27812,0.26587,0.17723,0.17466,0.19405
7,0.1,0.01,0.48638,0.48275,0.13708,0.28122,0.32425
8,0.1,0.01,0.41875,0.37538,0.11681,0.31075,79.04335
9,0.1,0.01,0.38843,0.42875,0.31244,0.32104,0.25826


Variance                         0.100000
Bias                             0.010000
Prior Model MAE                  0.385658
ABC pre-generator MAE            0.382770
Skip Node weight                 0.157999
ABC GAN MAE                      0.291414
ABC_GAN MAE (skip connection)    0.291257
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.35989,0.36228,0.12434,0.27899,0.16319
1,0.01,0.01,0.49100,0.48709,0.14930,0.47686,0.32685
2,0.01,0.01,0.31965,0.31279,0.14853,0.38471,324.35423
3,0.01,0.01,0.48614,0.48666,0.07869,0.36744,47.89028
4,0.01,0.01,0.44581,0.44646,0.18750,0.37733,0.27410
5,0.01,0.01,0.33082,0.33031,0.16819,0.30089,170.24077
6,0.01,0.01,0.34958,0.35190,0.12512,0.25503,122.50966
7,0.01,0.01,0.42027,0.41859,0.21003,0.29019,76.77868
8,0.01,0.01,0.43336,0.43338,0.12368,0.27799,0.33361
9,0.01,0.01,0.38277,0.38477,0.14843,0.24476,58.47750


Variance                          0.010000
Bias                              0.010000
Prior Model MAE                   0.401518
ABC pre-generator MAE             0.401684
Skip Node weight                  0.148480
ABC GAN MAE                       0.295540
ABC_GAN MAE (skip connection)    53.183891
dtype: float64
-------------------------------------------------------------


In [28]:
# Display Stats Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([gan_median,stats,paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['GAN','Stats','Variance','Bias','Prior Model','mGAN','skipGAN','Weight'])
display(df.round(5))

,Variance,Bias,Stats,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.37071,1.23929,0.17708,0.17066,0.18393
1,1.00,0.10,0.37071,1.06144,0.22779,0.27995,0.08081
2,1.00,0.01,0.37071,1.04272,0.27345,0.31993,0.07639
3,0.10,1.00,0.37071,0.90755,0.21637,0.18107,0.21889
4,0.10,0.10,0.37071,0.33531,0.26273,0.21763,0.09530
5,0.10,0.01,0.37071,0.34894,0.29566,0.33063,0.04127
6,0.01,1.00,0.37071,0.74672,0.17123,0.16486,0.14193
7,0.01,0.10,0.37071,0.29085,0.24500,0.24257,0.29860
8,0.01,0.01,0.37071,0.30009,0.27688,0.29899,0.02710
